In [1]:
from flask import Flask, request, jsonify, render_template, send_file
from pydub import AudioSegment
from aip import AipSpeech
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import os
from langchain_community.llms import Tongyi
from langchain_core.messages import HumanMessage,AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
import os

In [2]:
def convert_wav_to_wav(wav_file_path, output_wav_file_path):
    """将 WAV 文件转换为指定格式，并调整采样率、采样宽度、声道数和增益"""
    # 加载 WAV 文件
    audio = AudioSegment.from_wav(wav_file_path)

    # 设置采样率为16000 Hz
    audio = audio.set_frame_rate(16000)

    # 设置 2 字节采样（16-bit）
    audio = audio.set_sample_width(2)

    # 设置为单声道
    audio = audio.set_channels(1)

    # 增加 10dB
    audio = audio + 10

    # 导出为 WAV 文件，使用 PCM 编码
    audio.export(output_wav_file_path, format="wav", codec="pcm_s16le")

wav_file_path = './yun/uploadstalk/recording.wav'
output_wav_file_path = './yun/uploads/output.wav'
convert_wav_to_wav(wav_file_path, output_wav_file_path)

CouldntDecodeError: Decoding failed. ffmpeg returned error code: 3199971767

Output from ffmpeg/avlib:

ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enable-d3d12va --enable-ffnvcodec --enable-libvpl --enable-nvdec --enable-n

In [3]:
import subprocess

def convert_wav_to_wav(wav_file_path, output_wav_file_path):
    """直接使用 FFmpeg 转换 WAV 文件，并调整采样率、采样宽度、声道数和增益"""
    
    # FFmpeg 命令
    command = [
        "ffmpeg",
        "-i", wav_file_path,           # 输入文件
        "-ar", "16000",                 # 设置采样率为16000 Hz
        "-ac", "1",                     # 设置为单声道
        "-sample_fmt", "s16",           # 设置为 16-bit
        "-filter:a", "volume=10dB",     # 增加 10 dB
        output_wav_file_path            # 输出文件路径
    ]
    
    try:
        # 执行 FFmpeg 命令
        subprocess.run(command, check=True)
        print(f"文件已成功转换并保存为 {output_wav_file_path}")
    except subprocess.CalledProcessError as e:
        print(f"转换失败：{e}")

# 使用示例
wav_file_path = './yun/uploadstalk/recording.wav'
output_wav_file_path = './yun/uploads/output.wav'
convert_wav_to_wav(wav_file_path, output_wav_file_path)


文件已成功转换并保存为 ./yun/uploads/output.wav
